# Candidate Nightly Epochs

Author: Melissa Graham

This notebook uses as input the data files of "probably-real" candidates that were generated by `source_detection_summary.ipynb` and are stored in `source_detection_summary_files/`.

Keep in mind that only candidates with >10 objects (detections) and a mean R/B score > 0.4 were written to these files.

All of the objects (detections) for a given night in a given filter are combined for all candidates, with the result of one photometric point per filter per night.

## Set up 

In [ ]:
# import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# import pandas as pd

# from copy import deepcopy
import time

## Read the data files

#### all exposures

In [ ]:
fnm = 'source_detection_summary_files/exposures_cosmos.dat'
cexp_cal    = np.loadtxt( fnm, dtype='str',   usecols=(0) )
cexp_fbs    = np.loadtxt( fnm, dtype='str',   usecols=(1) )
cexp_id     = np.loadtxt( fnm, dtype='int',   usecols=(2) )
cexp_mjd    = np.loadtxt( fnm, dtype='float', usecols=(3) )
cexp_fil    = np.loadtxt( fnm, dtype='str',   usecols=(4) )
cexp_mlmg   = np.loadtxt( fnm, dtype='float', usecols=(5) )
cexp_msee   = np.loadtxt( fnm, dtype='float', usecols=(6) )
cexp_msks   = np.loadtxt( fnm, dtype='float', usecols=(7) )
cexp_tnobj  = np.loadtxt( fnm, dtype='int',   usecols=(8) )
cexp_tnobjg = np.loadtxt( fnm, dtype='int',   usecols=(9) )
del fnm

fnm = 'source_detection_summary_files/exposures_elais.dat'
eexp_cal    = np.loadtxt( fnm, dtype='str',   usecols=(0) )
eexp_fbs    = np.loadtxt( fnm, dtype='str',   usecols=(1) )
eexp_id     = np.loadtxt( fnm, dtype='int',   usecols=(2) )
eexp_mjd    = np.loadtxt( fnm, dtype='float', usecols=(3) )
eexp_fil    = np.loadtxt( fnm, dtype='str',   usecols=(4) )
eexp_mlmg   = np.loadtxt( fnm, dtype='float', usecols=(5) )
eexp_msee   = np.loadtxt( fnm, dtype='float', usecols=(6) )
eexp_msks   = np.loadtxt( fnm, dtype='float', usecols=(7) )
eexp_tnobj  = np.loadtxt( fnm, dtype='int',   usecols=(8) )
eexp_tnobjg = np.loadtxt( fnm, dtype='int',   usecols=(9) )
del fnm

#### create arrays for the unique epochs from the exposures

In [ ]:
c_u_cal = np.unique( np.sort( cexp_cal ) )
e_u_cal = np.unique( np.sort( eexp_cal ) )
print( 'Number of unique epochs for COSMOS and ELAIS: ',len(c_u_cal),len(e_u_cal) )

c_u_cal_meanmjd = np.zeros( len(c_u_cal), dtype='float' )
e_u_cal_meanmjd = np.zeros( len(e_u_cal), dtype='float' )
c_u_cal_meanlmg = np.zeros( len(c_u_cal), dtype='float' )
e_u_cal_meanlmg = np.zeros( len(e_u_cal), dtype='float' )

for c in range(len(c_u_cal)):
    tx = np.where( cexp_cal == c_u_cal[c] )[0]
    c_u_cal_meanmjd[c] = np.mean( cexp_mjd[tx] )
    c_u_cal_meanlmg[c] = np.mean( cexp_mlmg[tx] )
    del tx

for e in range(len(e_u_cal)):
    tx = np.where( eexp_cal == e_u_cal[e] )[0]
    e_u_cal_meanmjd[e] = np.mean( eexp_mjd[tx] )
    e_u_cal_meanlmg[e] = np.mean( eexp_mlmg[tx] )
    del tx

# print(c_u_cal_meanmjd)
# print(e_u_cal_meanmjd)

#### all candidates

In [ ]:
fnm = 'source_detection_summary_files/candidates.dat'
cand_field    = np.loadtxt( fnm, dtype='str',   usecols=(0) )
cand_id       = np.loadtxt( fnm, dtype='str',   usecols=(1) )
cand_ra       = np.loadtxt( fnm, dtype='float', usecols=(2) )
cand_dec      = np.loadtxt( fnm, dtype='float', usecols=(3) )
cand_nobj     = np.loadtxt( fnm, dtype='int',   usecols=(4) )
cand_meanrb   = np.loadtxt( fnm, dtype='float', usecols=(5) )
cand_stdrb    = np.loadtxt( fnm, dtype='float', usecols=(6) )
cand_tspan    = np.loadtxt( fnm, dtype='float', usecols=(7) )
cand_lcamp    = np.loadtxt( fnm, dtype='float', usecols=(8) )
cand_nobj_g   = np.loadtxt( fnm, dtype='int',   usecols=(9) )
cand_meanrb_g = np.loadtxt( fnm, dtype='float', usecols=(10) )
cand_stdrb_g  = np.loadtxt( fnm, dtype='float', usecols=(11) )
cand_tspan_g  = np.loadtxt( fnm, dtype='float', usecols=(12) )
cand_lcamp_g  = np.loadtxt( fnm, dtype='float', usecols=(13) )
cand_nobj_r   = np.loadtxt( fnm, dtype='int',   usecols=(14) )
cand_meanrb_r = np.loadtxt( fnm, dtype='float', usecols=(15) )
cand_stdrb_r  = np.loadtxt( fnm, dtype='float', usecols=(16) )
cand_tspan_r  = np.loadtxt( fnm, dtype='float', usecols=(17) )
cand_lcamp_r  = np.loadtxt( fnm, dtype='float', usecols=(18) )
cand_nobj_i   = np.loadtxt( fnm, dtype='int',   usecols=(19) )
cand_meanrb_i = np.loadtxt( fnm, dtype='float', usecols=(20) )
cand_stdrb_i  = np.loadtxt( fnm, dtype='float', usecols=(21) )
cand_tspan_i  = np.loadtxt( fnm, dtype='float', usecols=(22) )
cand_lcamp_i  = np.loadtxt( fnm, dtype='float', usecols=(23) )
del fnm

print('Total number of candidates: ',len(cand_id))

#### candidates' objects data ("detections")

In [ ]:
fnm = 'source_detection_summary_files/candidate_objects_cosmos.dat'
clc_candid  = np.loadtxt( fnm, dtype='str',   usecols=(0) )
clc_objid   = np.loadtxt( fnm, dtype='str',   usecols=(1) )
clc_objra   = np.loadtxt( fnm, dtype='float', usecols=(2) )
clc_objdec  = np.loadtxt( fnm, dtype='float', usecols=(3) )
clc_objmjd  = np.loadtxt( fnm, dtype='float', usecols=(4) )
clc_objfil  = np.loadtxt( fnm, dtype='str',   usecols=(5) )
clc_objmag  = np.loadtxt( fnm, dtype='float', usecols=(6) )
clc_objmage = np.loadtxt( fnm, dtype='float', usecols=(7) )
clc_objrb   = np.loadtxt( fnm, dtype='float', usecols=(8) )
del fnm

fnm = 'source_detection_summary_files/candidate_objects_elais.dat'
elc_candid  = np.loadtxt( fnm, dtype='str',   usecols=(0) )
elc_objid   = np.loadtxt( fnm, dtype='str',   usecols=(1) )
elc_objra   = np.loadtxt( fnm, dtype='float', usecols=(2) )
elc_objdec  = np.loadtxt( fnm, dtype='float', usecols=(3) )
elc_objmjd  = np.loadtxt( fnm, dtype='float', usecols=(4) )
elc_objfil  = np.loadtxt( fnm, dtype='str',   usecols=(5) )
elc_objmag  = np.loadtxt( fnm, dtype='float', usecols=(6) )
elc_objmage = np.loadtxt( fnm, dtype='float', usecols=(7) )
elc_objrb   = np.loadtxt( fnm, dtype='float', usecols=(8) )
del fnm

## Combine the photometry per night, per filter

Include non-detections and limiting magnitudes, from the exposures arrays.

In [ ]:
t1 = time.time()

temp_clcNE_candid = []
temp_clcNE_cal = []
temp_clcNE_mjd = []
temp_clcNE_fil = []
temp_clcNE_nobj = []
temp_clcNE_mag = []
temp_clcNE_mage = []
temp_clcNE_mrb = []
temp_clcNE_lmag = []

temp_elcNE_candid = []
temp_elcNE_cal = []
temp_elcNE_mjd = []
temp_elcNE_fil = []
temp_elcNE_nobj = []
temp_elcNE_mag = []
temp_elcNE_mage = []
temp_elcNE_mrb = []
temp_elcNE_lmag = []

### for every candidate
for c in range(len(cand_field)):
    if (c == 10) | (c == 100) | (c == 1000):
        t2 = time.time()
        print(c, ((t2-t1)/float(c))*(float(len(cand_field)-c)),' remain')
    
    if cand_field[c] == 'COSMOS':
        cx = np.where(clc_candid == cand_id[c])[0]
        for d,mjd in enumerate(c_u_cal_meanmjd):
            dx = np.where(np.abs(clc_objmjd[cx] - c_u_cal_meanmjd[d]) < 0.3)[0]
            for f,fil in enumerate(['g','r','i']):
                fx = np.where(clc_objfil[cx[dx]] == fil)[0]
                tx = cx[dx[fx]]
                if len(tx) >= 2:
                    temp_clcNE_candid.append( cand_id[c] )
                    temp_clcNE_cal.append( c_u_cal[d] )
                    temp_clcNE_mjd.append( np.mean(clc_objmjd[tx]) )
                    temp_clcNE_fil.append( fil )
                    temp_clcNE_nobj.append( len(tx) )
                    temp_clcNE_mag.append( np.mean(clc_objmag[tx]) )
                    temp_clcNE_mage.append( np.sqrt(np.mean(clc_objmage[tx])**2 + \
                                                    np.std(clc_objmag[tx])**2) )
                    temp_clcNE_mrb.append( np.mean(clc_objrb[tx]) )
                    temp_clcNE_lmag.append( c_u_cal_meanlmg[d] )
                elif len(tx) == 1:
                    temp_clcNE_candid.append( cand_id[c] )
                    temp_clcNE_cal.append( c_u_cal[d] )
                    temp_clcNE_mjd.append( clc_objmjd[tx[0]] )
                    temp_clcNE_fil.append( fil )
                    temp_clcNE_nobj.append( 1 )
                    temp_clcNE_mag.append( clc_objmag[tx[0]] )
                    temp_clcNE_mage.append( clc_objmage[tx[0]] )
                    temp_clcNE_mrb.append( clc_objrb[tx[0]] )
                    temp_clcNE_lmag.append( c_u_cal_meanlmg[d] )
                elif len(tx) == 0:
                    temp_clcNE_candid.append( cand_id[c] )
                    temp_clcNE_cal.append( c_u_cal[d] )
                    temp_clcNE_mjd.append( c_u_cal_meanmjd[d] )
                    temp_clcNE_fil.append( fil )
                    temp_clcNE_nobj.append( 0 )
                    temp_clcNE_mag.append( float('NaN') )
                    temp_clcNE_mage.append( float('NaN') )
                    temp_clcNE_mrb.append( float('NaN') )
                    temp_clcNE_lmag.append( c_u_cal_meanlmg[d] )
                del tx,fx
            del dx
        del cx

    if cand_field[c] == 'ELAIS':
        cx = np.where(elc_candid == cand_id[c])[0]
        for d,mjd in enumerate(e_u_cal_meanmjd):
            dx = np.where(np.abs(elc_objmjd[cx] - e_u_cal_meanmjd[d]) < 0.3)[0]
            for f,fil in enumerate(['g','r','i']):
                fx = np.where(elc_objfil[cx[dx]] == fil)[0]
                tx = cx[dx[fx]]
                if len(tx) >= 2:
                    temp_elcNE_candid.append( cand_id[c] )
                    temp_elcNE_cal.append( e_u_cal[d] )
                    temp_elcNE_mjd.append( np.mean(elc_objmjd[tx]) )
                    temp_elcNE_fil.append( fil )
                    temp_elcNE_nobj.append( len(tx) )
                    temp_elcNE_mag.append( np.mean(elc_objmag[tx]) )
                    temp_elcNE_mage.append( np.sqrt(np.mean(elc_objmage[tx])**2 + \
                                                    np.std(elc_objmag[tx])**2) )
                    temp_elcNE_mrb.append( np.mean(elc_objrb[tx]) )
                    temp_elcNE_lmag.append( e_u_cal_meanlmg[d] )
                elif len(tx) == 1:
                    temp_elcNE_candid.append( cand_id[c] )
                    temp_elcNE_cal.append( e_u_cal[d] )
                    temp_elcNE_mjd.append( np.mean(elc_objmjd[tx]) )
                    temp_elcNE_fil.append( fil )
                    temp_elcNE_nobj.append( 1 )
                    temp_elcNE_mag.append( elc_objmag[tx[0]] )
                    temp_elcNE_mage.append( elc_objmage[tx[0]] )
                    temp_elcNE_mrb.append( elc_objrb[tx[0]] )
                    temp_elcNE_lmag.append( e_u_cal_meanlmg[d] )
                elif len(tx) == 0:
                    temp_elcNE_candid.append( cand_id[c] )
                    temp_elcNE_cal.append( e_u_cal[d] )
                    temp_elcNE_mjd.append( e_u_cal_meanmjd[d] )
                    temp_elcNE_fil.append( fil )
                    temp_elcNE_nobj.append( 0 )
                    temp_elcNE_mag.append( float('NaN') )
                    temp_elcNE_mage.append( float('NaN') )
                    temp_elcNE_mrb.append( float('NaN') )
                    temp_elcNE_lmag.append( e_u_cal_meanlmg[d] )                    
                del tx,fx
            del dx
        del cx

clcNE_candid = np.asarray( temp_clcNE_candid, dtype='str' )
clcNE_nobj   = np.asarray( temp_clcNE_nobj, dtype='int' )
clcNE_mjd    = np.asarray( temp_clcNE_mjd, dtype='float' )
clcNE_cal    = np.asarray( temp_clcNE_cal, dtype='str' )
clcNE_fil    = np.asarray( temp_clcNE_fil, dtype='str' )
clcNE_mag    = np.asarray( temp_clcNE_mag, dtype='float' )
clcNE_mage   = np.asarray( temp_clcNE_mage, dtype='float' )
clcNE_mrb    = np.asarray( temp_clcNE_mrb, dtype='float' )
clcNE_lmag   = np.asarray( temp_clcNE_lmag, dtype='float' )

elcNE_candid = np.asarray( temp_elcNE_candid, dtype='str' )
elcNE_nobj   = np.asarray( temp_elcNE_nobj, dtype='int' )
elcNE_mjd    = np.asarray( temp_elcNE_mjd, dtype='float' )
elcNE_cal    = np.asarray( temp_elcNE_cal, dtype='str' )
elcNE_fil    = np.asarray( temp_elcNE_fil, dtype='str' )
elcNE_mag    = np.asarray( temp_elcNE_mag, dtype='float' )
elcNE_mage   = np.asarray( temp_elcNE_mage, dtype='float' )
elcNE_mrb    = np.asarray( temp_elcNE_mrb, dtype='float' )
elcNE_lmag   = np.asarray( temp_elcNE_lmag, dtype='float' )

del temp_clcNE_candid,temp_clcNE_nobj,temp_clcNE_mjd,temp_clcNE_cal
del temp_clcNE_fil,temp_clcNE_mag,temp_clcNE_mage,temp_clcNE_mrb,temp_clcNE_lmag
del temp_elcNE_candid,temp_elcNE_nobj,temp_elcNE_mjd,temp_elcNE_cal
del temp_elcNE_fil,temp_elcNE_mag,temp_elcNE_mage,temp_elcNE_mrb,temp_elcNE_lmag

t2 = time.time()
print('elapsed: ',t2-t1)

### write nightly epoch candidate photometry to file

In [ ]:
fnm = 'candidate_nightly_epochs_files/candidate_lightcurves.dat'
fout = open(fnm, 'w')

fout.write('# Melissa Graham, candidate_nightly_epochs.ipynb \n')
tnow = time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.gmtime())
fout.write('# UTC = '+tnow+' \n')
del tnow
fout.write('# \n')
fout.write('# Columns \n')
fout.write('#  0 field  \n')
fout.write('#  1 id -- candidate identifier \n')
fout.write('#  2 calendar date \n')
fout.write('#  3 MJD \n')
fout.write('#  4 filter \n')
fout.write('#  5 number of objects combined \n')
fout.write('#  6 magnitude (mean of objects combined) \n')
fout.write('#  7 magnitude error \n')
fout.write('#  8 real/bogus (mean of objects combined) \n')
fout.write('#  9 exposure limiting magnitude (nightly mean) \n')
fout.write('# \n')

for i in range(len(clcNE_candid)):
    fout.write('COSMOS %-9s %-8s %12.6f '\
               '%1s %3i %6.3f %6.3f %6.4f %6.3f \n' % \
               (clcNE_candid[i], clcNE_cal[i], clcNE_mjd[i], \
                clcNE_fil[i], clcNE_nobj[i], clcNE_mag[i], clcNE_mage[i], clcNE_mrb[i], clcNE_lmag[i]) )

for i in range(len(elcNE_candid)):
    fout.write('ELAIS %-9s %-8s %12.6f '\
               '%1s %3i %6.3f %6.3f %6.4f %6.3f \n' % \
               (elcNE_candid[i], elcNE_cal[i], elcNE_mjd[i], \
                elcNE_fil[i], elcNE_nobj[i], elcNE_mag[i], elcNE_mage[i], elcNE_mrb[i], elcNE_lmag[i]) )

fout.close()
print('Wrote to: ',fnm)
del fnm

## Create summary parameters for nightly-epoch lightcurves

Timespan (total duration, from first to last detection)

Amplitude (difference between brightest and faintest detection)

Number of epochs

In [ ]:
cand_tspanNE = np.zeros( len(cand_field), dtype='float' )
cand_tspanNE_g = np.zeros( len(cand_field), dtype='float' )
cand_tspanNE_r = np.zeros( len(cand_field), dtype='float' )
cand_tspanNE_i = np.zeros( len(cand_field), dtype='float' )

cand_lcampNE = np.zeros( len(cand_field), dtype='float' )
cand_lcampNE_g = np.zeros( len(cand_field), dtype='float' )
cand_lcampNE_r = np.zeros( len(cand_field), dtype='float' )
cand_lcampNE_i = np.zeros( len(cand_field), dtype='float' )

cand_numNE = np.zeros( len(cand_field), dtype='int' )
cand_numNE_g = np.zeros( len(cand_field), dtype='int' )
cand_numNE_r = np.zeros( len(cand_field), dtype='int' )
cand_numNE_i = np.zeros( len(cand_field), dtype='int' )

### for every candidate
for c in range(len(cand_field)):
    if cand_field[c] == 'COSMOS':
        tx = np.where( (clcNE_candid == cand_id[c]) & (clcNE_nobj >= 1) )[0]
        temp = np.unique( clcNE_cal[tx] )
        cand_numNE[c] = len(temp)
        del temp
        cand_tspanNE[c] = np.nanmax( clcNE_mjd[tx] ) - np.nanmin( clcNE_mjd[tx] )
        cand_lcampNE[c] = np.nanmax( clcNE_mag[tx] ) - np.nanmin( clcNE_mag[tx] )
        del tx
        
        gx = np.where( (clcNE_candid == cand_id[c]) & (clcNE_nobj >= 1) & (clcNE_fil == 'g') )[0]
        rx = np.where( (clcNE_candid == cand_id[c]) & (clcNE_nobj >= 1) & (clcNE_fil == 'r') )[0]
        ix = np.where( (clcNE_candid == cand_id[c]) & (clcNE_nobj >= 1) & (clcNE_fil == 'i') )[0]
        if len(gx) > 0:
            cand_numNE_g[c] = len(gx)
            cand_tspanNE_g[c] = np.nanmax( clcNE_mjd[gx] ) - np.nanmin( clcNE_mjd[gx] )
            cand_lcampNE_g[c] = np.nanmax( clcNE_mag[gx] ) - np.nanmin( clcNE_mag[gx] )
        if len(rx) > 0:
            cand_numNE_r[c] = len(rx)
            cand_tspanNE_r[c] = np.nanmax( clcNE_mjd[rx] ) - np.nanmin( clcNE_mjd[rx] )
            cand_lcampNE_r[c] = np.nanmax( clcNE_mag[rx] ) - np.nanmin( clcNE_mag[rx] )
        if len(ix) > 0:
            cand_numNE_i[c] = len(ix)
            cand_tspanNE_i[c] = np.nanmax( clcNE_mjd[ix] ) - np.nanmin( clcNE_mjd[ix] )
            cand_lcampNE_i[c] = np.nanmax( clcNE_mag[ix] ) - np.nanmin( clcNE_mag[ix] )
        del gx,rx,ix
    
    if cand_field[c] == 'ELAIS':
        tx = np.where( (elcNE_candid == cand_id[c]) & (elcNE_nobj >= 1) )[0]
        temp = np.unique( elcNE_cal[tx] )
        cand_numNE[c] = len(temp)
        del temp
        cand_tspanNE[c] = np.nanmax( elcNE_mjd[tx] ) - np.nanmin( elcNE_mjd[tx] )
        cand_lcampNE[c] = np.nanmax( elcNE_mag[tx] ) - np.nanmin( elcNE_mag[tx] )
        del tx
        
        gx = np.where( (elcNE_candid == cand_id[c]) & (elcNE_nobj >= 1) & (elcNE_fil == 'g') )[0]
        rx = np.where( (elcNE_candid == cand_id[c]) & (elcNE_nobj >= 1) & (elcNE_fil == 'r') )[0]
        ix = np.where( (elcNE_candid == cand_id[c]) & (elcNE_nobj >= 1) & (elcNE_fil == 'i') )[0]
        if len(gx) > 0:
            cand_numNE_g[c] = len(gx)
            cand_tspanNE_g[c] = np.nanmax( elcNE_mjd[gx] ) - np.nanmin( elcNE_mjd[gx] )
            cand_lcampNE_g[c] = np.nanmax( elcNE_mag[gx] ) - np.nanmin( elcNE_mag[gx] )
        if len(rx) > 0:
            cand_numNE_r[c] = len(rx)
            cand_tspanNE_r[c] = np.nanmax( elcNE_mjd[rx] ) - np.nanmin( elcNE_mjd[rx] )
            cand_lcampNE_r[c] = np.nanmax( elcNE_mag[rx] ) - np.nanmin( elcNE_mag[rx] )
        if len(ix) > 0:
            cand_numNE_i[c] = len(ix)
            cand_tspanNE_i[c] = np.nanmax( elcNE_mjd[ix] ) - np.nanmin( elcNE_mjd[ix] )
            cand_lcampNE_i[c] = np.nanmax( elcNE_mag[ix] ) - np.nanmin( elcNE_mag[ix] )
        del gx,rx,ix

### Write lightcurve parameters to file

In [ ]:
fnm = 'candidate_nightly_epochs_files/candidate_lightcurve_parameters.dat'
fout = open(fnm, 'w')

fout.write('# Melissa Graham, candidate_nightly_epochs.ipynb \n')
tnow = time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.gmtime())
fout.write('# UTC = '+tnow+' \n')
del tnow
fout.write('# \n')
fout.write('# Columns \n')
fout.write('#  0 field  \n')
fout.write('#  1 id -- candidate identifier \n')
fout.write('#  2 timespan (days between first and last detection) \n')
fout.write('#  3 timespan in g \n')
fout.write('#  4 timespan in r \n')
fout.write('#  5 timespan in i \n')
fout.write('#  6 amplitude (magnitudes between brightest and faintest detection) \n')
fout.write('#  7 amplitude g \n')
fout.write('#  8 amplitude r \n')
fout.write('#  9 amplitude i \n')
fout.write('#  0 number of epochs (number of unique nights detected) \n')
fout.write('# 10 number of epochs in g \n')
fout.write('# 11 number of epochs in r \n')
fout.write('# 12 number of epochs in i \n')
fout.write('# \n')

for i in range(len(cand_field)):
    fout.write('%-8s %-9s %7.2f %7.2f %7.2f %7.2f %6.3f %6.3f %6.3f %6.3f %3i %3i %3i %3i \n' % \
               (cand_field[i], cand_id[i], \
                cand_tspanNE[i], cand_tspanNE_g[i], cand_tspanNE_r[i], cand_tspanNE_i[i], \
                cand_lcampNE[i], cand_lcampNE_g[i], cand_lcampNE_r[i], cand_lcampNE_i[i], \
                cand_numNE[i], cand_numNE_g[i], cand_numNE_r[i], cand_numNE_i[i] ) )

fout.close()
print('Wrote to: ',fnm)
del fnm